In [116]:
import pandas as pd
import numpy as np

In [117]:
column_names = ['Ticker','Date_YMD','Time','open','high','low','close','volume','extra']
df = pd.read_csv('../nifty-banknifty-data/Nifty-index-data/2021/Yearly Consolidated/NIFTY.csv',names=column_names,header=None)

df


,Ticker,Date_YMD,Time,open,high,low,close,volume,extra
0,NIFTY,2021/01/01,09:16,13997.9,14020.8,13991.3,14013.2,0,0
1,NIFTY,2021/01/01,09:17,14014.8,14018.5,14008.2,14009.0,0,0
2,NIFTY,2021/01/01,09:18,14008.0,14013.1,14005.0,14012.7,0,0
3,NIFTY,2021/01/01,09:19,14013.7,14019.1,14013.7,14016.2,0,0
4,NIFTY,2021/01/01,09:20,14015.5,14017.8,14012.0,14015.5,0,0
...,...,...,...,...,...,...,...,...,...
92253,NIFTY,2021/12/31,15:26,17357.9,17358.5,17353.8,17356.3,0,0
92254,NIFTY,2021/12/31,15:27,17356.1,17359.7,17355.6,17359.3,0,0
92255,NIFTY,2021/12/31,15:28,17358.8,17360.5,17356.2,17359.8,0,0
92256,NIFTY,2021/12/31,15:29,17359.8,17360.9,17351.1,17353.6,0,0


In [118]:
df.drop(columns=['volume','extra'],inplace=True)
df



,Ticker,Date_YMD,Time,open,high,low,close
0,NIFTY,2021/01/01,09:16,13997.9,14020.8,13991.3,14013.2
1,NIFTY,2021/01/01,09:17,14014.8,14018.5,14008.2,14009.0
2,NIFTY,2021/01/01,09:18,14008.0,14013.1,14005.0,14012.7
3,NIFTY,2021/01/01,09:19,14013.7,14019.1,14013.7,14016.2
4,NIFTY,2021/01/01,09:20,14015.5,14017.8,14012.0,14015.5
...,...,...,...,...,...,...,...
92253,NIFTY,2021/12/31,15:26,17357.9,17358.5,17353.8,17356.3
92254,NIFTY,2021/12/31,15:27,17356.1,17359.7,17355.6,17359.3
92255,NIFTY,2021/12/31,15:28,17358.8,17360.5,17356.2,17359.8
92256,NIFTY,2021/12/31,15:29,17359.8,17360.9,17351.1,17353.6


In [119]:
# x = df.where((df['Time'] == '09:16') | (df['Time'] == '15:30'))
# x = x.dropna()
x = df.copy()
x['DateTime'] = pd.to_datetime(x['Date_YMD'] + ' ' + x['Time'])
x.set_index('DateTime', inplace=True)
resampled_df = x.resample('15min').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last'
})
# print(resampled_df.shape)
resampled_df.dropna(inplace=True)
temp = resampled_df.loc['2021-08-02 09:15:00':'2021-08-02 15:30:00',:]
print(len(temp))

resampled_df

26


,open,high,low,close
DateTime,,,,
2021-01-01 09:15:00,13997.9,14023.5,13991.3,14018.5
2021-01-01 09:30:00,14018.0,14028.9,14001.2,14024.0
2021-01-01 09:45:00,14024.2,14033.8,14022.5,14032.2
2021-01-01 10:00:00,14032.3,14033.3,14016.3,14025.8
2021-01-01 10:15:00,14025.0,14036.5,14017.6,14035.5
...,...,...,...,...
2021-12-31 14:30:00,17353.5,17369.3,17351.4,17368.8
2021-12-31 14:45:00,17369.3,17377.3,17358.1,17362.8
2021-12-31 15:00:00,17363.5,17365.2,17341.8,17356.6


In [120]:
new_df = resampled_df.copy()

#Heiken Ashi Close
new_df['HA_Close'] = (new_df['open'] + new_df['close'] + new_df['high'] + new_df['low'])/4


#Heiken Ashi Open
new_df['HA_Open'] = pd.Series([float('nan') * len(new_df)])
new_df.iloc[0,new_df.columns.get_loc('HA_Open')] = (new_df.iloc[0,new_df.columns.get_loc('open')] + new_df.iloc[0,new_df.columns.get_loc('close')]) / 2

for i in range(1, len(new_df)):
    new_df.iloc[i, new_df.columns.get_loc('HA_Open')] = (new_df.iloc[i - 1]['HA_Open'] + new_df.iloc[i - 1]['HA_Close']) / 2


#Heiken Ashi High
new_df['HA_High'] = new_df[['high', 'HA_Open', 'HA_Close']].max(axis=1)

#Heiken Ashi Low
new_df['HA_Low'] = new_df[['low', 'HA_Open', 'HA_Close']].min(axis=1)

new_df


,open,high,low,close,HA_Close,HA_Open,HA_High,HA_Low
DateTime,,,,,,,,
2021-01-01 09:15:00,13997.9,14023.5,13991.3,14018.5,14007.800,14008.200000,14023.5,13991.3000
2021-01-01 09:30:00,14018.0,14028.9,14001.2,14024.0,14018.025,14008.000000,14028.9,14001.2000
2021-01-01 09:45:00,14024.2,14033.8,14022.5,14032.2,14028.175,14013.012500,14033.8,14013.0125
2021-01-01 10:00:00,14032.3,14033.3,14016.3,14025.8,14026.925,14020.593750,14033.3,14016.3000
2021-01-01 10:15:00,14025.0,14036.5,14017.6,14035.5,14028.650,14023.759375,14036.5,14017.6000
...,...,...,...,...,...,...,...,...
2021-12-31 14:30:00,17353.5,17369.3,17351.4,17368.8,17360.750,17363.481548,17369.3,17351.4000
2021-12-31 14:45:00,17369.3,17377.3,17358.1,17362.8,17366.875,17362.115774,17377.3,17358.1000
2021-12-31 15:00:00,17363.5,17365.2,17341.8,17356.6,17356.775,17364.495387,17365.2,17341.8000


In [121]:
new_df.drop(['open','high','low','close'],axis=1,inplace=True)
new_df.columns = ['close','open','high','low']
new_df



,close,open,high,low
DateTime,,,,
2021-01-01 09:15:00,14007.800,14008.200000,14023.5,13991.3000
2021-01-01 09:30:00,14018.025,14008.000000,14028.9,14001.2000
2021-01-01 09:45:00,14028.175,14013.012500,14033.8,14013.0125
2021-01-01 10:00:00,14026.925,14020.593750,14033.3,14016.3000
2021-01-01 10:15:00,14028.650,14023.759375,14036.5,14017.6000
...,...,...,...,...
2021-12-31 14:30:00,17360.750,17363.481548,17369.3,17351.4000
2021-12-31 14:45:00,17366.875,17362.115774,17377.3,17358.1000
2021-12-31 15:00:00,17356.775,17364.495387,17365.2,17341.8000


In [122]:
new_df['type'] = new_df.apply(lambda row: 'call' if row['close'] > row['open'] else 'put', axis=1)

new_df

,close,open,high,low,type
DateTime,,,,,
2021-01-01 09:15:00,14007.800,14008.200000,14023.5,13991.3000,put
2021-01-01 09:30:00,14018.025,14008.000000,14028.9,14001.2000,call
2021-01-01 09:45:00,14028.175,14013.012500,14033.8,14013.0125,call
2021-01-01 10:00:00,14026.925,14020.593750,14033.3,14016.3000,call
2021-01-01 10:15:00,14028.650,14023.759375,14036.5,14017.6000,call
...,...,...,...,...,...
2021-12-31 14:30:00,17360.750,17363.481548,17369.3,17351.4000,put
2021-12-31 14:45:00,17366.875,17362.115774,17377.3,17358.1000,call
2021-12-31 15:00:00,17356.775,17364.495387,17365.2,17341.8000,put


In [123]:

def check_time(index):
    return (index.time() == pd.Timestamp('09:15:00').time()) | (index.time() == pd.Timestamp('15:15:00').time())

mask = new_df.index.map(check_time)

flc = new_df[mask]
flc

,close,open,high,low,type
DateTime,,,,,
2021-01-01 09:15:00,14007.800,14008.200000,14023.500000,13991.300000,put
2021-01-01 15:15:00,14015.425,14035.185230,14035.185230,14001.600000,put
2021-01-04 09:15:00,14099.375,14019.965058,14107.800000,14019.965058,call
2021-01-04 15:15:00,14134.975,14115.239597,14148.000000,14115.239597,call
2021-01-05 09:15:00,14086.925,14134.291149,14134.291149,14048.200000,put
...,...,...,...,...,...
2021-12-29 15:15:00,17201.250,17239.863031,17239.863031,17176.700000,put
2021-12-30 09:15:00,17191.950,17203.290758,17217.100000,17146.300000,put
2021-12-30 15:15:00,17215.425,17212.681670,17226.300000,17197.800000,call


In [124]:
def calculate_difference(row):
    previous_row = flc.shift(1).loc[row.name]  # Get the previous row using shift() and loc[]
    if pd.isnull(previous_row).any():  # Check if the previous row exists (for the first row)
        return np.NAN
    else:
        return row['type'] ==  previous_row['type']

# Apply the function to each row
flc['result'] = flc.apply(calculate_difference, axis=1)
flc

/tmp/ipykernel_5528/520706145.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flc['result'] = flc.apply(calculate_difference, axis=1)


,close,open,high,low,type,result
DateTime,,,,,,
2021-01-01 09:15:00,14007.800,14008.200000,14023.500000,13991.300000,put,NaN
2021-01-01 15:15:00,14015.425,14035.185230,14035.185230,14001.600000,put,True
2021-01-04 09:15:00,14099.375,14019.965058,14107.800000,14019.965058,call,False
2021-01-04 15:15:00,14134.975,14115.239597,14148.000000,14115.239597,call,True
2021-01-05 09:15:00,14086.925,14134.291149,14134.291149,14048.200000,put,False
...,...,...,...,...,...,...
2021-12-29 15:15:00,17201.250,17239.863031,17239.863031,17176.700000,put,True
2021-12-30 09:15:00,17191.950,17203.290758,17217.100000,17146.300000,put,True
2021-12-30 15:15:00,17215.425,17212.681670,17226.300000,17197.800000,call,False


In [125]:
flc_t = flc[flc['result'] == True].copy()
flc_t

,close,open,high,low,type,result
DateTime,,,,,,
2021-01-01 15:15:00,14015.425,14035.185230,14035.185230,14001.600000,put,True
2021-01-04 15:15:00,14134.975,14115.239597,14148.000000,14115.239597,call,True
2021-01-06 09:15:00,14226.975,14194.498491,14244.200000,14194.498491,call,True
2021-01-06 15:15:00,14138.725,14135.751806,14157.900000,14124.000000,call,True
2021-01-07 09:15:00,14238.550,14134.056702,14256.300000,14134.056702,call,True
...,...,...,...,...,...,...
2021-12-28 09:15:00,17184.450,17091.925571,17203.900000,17091.925571,call,True
2021-12-28 15:15:00,17234.675,17211.931668,17242.700000,17211.931668,call,True
2021-12-29 15:15:00,17201.250,17239.863031,17239.863031,17176.700000,put,True


In [126]:
mask = flc_t.index.map(lambda x : x.time() == pd.Timestamp('09:15:00').time())
flc_t[mask]

,close,open,high,low,type,result
DateTime,,,,,,
2021-01-06 09:15:00,14226.975,14194.498491,14244.200000,14194.498491,call,True
2021-01-07 09:15:00,14238.550,14134.056702,14256.300000,14134.056702,call,True
2021-01-11 09:15:00,14439.825,14341.268994,14458.700000,14341.268994,call,True
2021-01-18 09:15:00,14426.325,14434.590471,14459.200000,14389.200000,put,True
2021-01-22 09:15:00,14575.925,14623.403746,14623.403746,14545.100000,put,True
...,...,...,...,...,...,...
2021-12-24 09:15:00,17113.600,17060.955702,17155.600000,17051.400000,call,True
2021-12-27 09:15:00,16912.675,17005.284209,17005.284209,16833.200000,put,True
2021-12-28 09:15:00,17184.450,17091.925571,17203.900000,17091.925571,call,True


In [127]:
import os

def list_files(directory):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_list.append(os.path.join(root, file))
    return file_list

# Example usage:
folder_path = "/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data"
files = list_files(folder_path)
files

['/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data/IEOD CSV.format',
 '/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data/2014/Yearly Consolidated/NIFTY.csv',
 '/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data/2014/Yearly Consolidated/BANKNIFTY.csv',
 '/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data/2014/Yearly Consolidated/INDIAVIX.csv',
 '/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data/2014/Yearly Consolidated/NIFTY_F1.csv',
 '/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data/2014/Yearly Consolidated/BANKNIFTY_F1.csv',
 '/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data/2014/Monthly Segregated/February/NIFTY.csv',
 '/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data/2014/Monthly Segregated/February/BANKNIFTY.csv',
 '/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data/2014/Monthly Segregated/February/INDIAVIX.csv',
 '/home/

In [129]:
import os

def find_files(directory, filename):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file == filename:
                file_list.append(os.path.join(root, file))
    return file_list

# Example usage:
folder_path = "/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data"
filename = "NIFTY.csv"
files = find_files(folder_path, filename)
files

['/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data/2014/Yearly Consolidated/NIFTY.csv',
 '/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data/2014/Monthly Segregated/February/NIFTY.csv',
 '/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data/2014/Monthly Segregated/August/NIFTY.csv',
 '/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data/2014/Monthly Segregated/July/NIFTY.csv',
 '/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data/2014/Monthly Segregated/November/NIFTY.csv',
 '/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data/2014/Monthly Segregated/December/NIFTY.csv',
 '/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data/2014/Monthly Segregated/May/NIFTY.csv',
 '/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data/2014/Monthly Segregated/April/NIFTY.csv',
 '/home/yashginoya/yash/Trading/nifty-banknifty-data/Nifty-index-data/2014/Monthly Segregated/Septembe